In [1]:
#Optional step to delete session if you get an error that multiple sessions are open
del session

NameError: name 'session' is not defined

# Initialize Notebook, import libraries and create Snowflake connection

In [16]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year
import streamlit as st
import pandas as pd
from datetime import date

# scikit-learn (install: pip install -U scikit-learn)
from sklearn.linear_model import LinearRegression

# Session
connection_parameters = {
   "account": "hq04573.ap-southeast-1",
   "user": "anna21",
   "password": "Bombay@123",
   "warehouse": "compute_wh",
   "role": "accountadmin",
   "database": "summit_hol",
   "schema": "public"
}
session = Session.builder.configs(connection_parameters).create()


# test if we have a connection
session.sql("select current_warehouse() wh, current_database() db, current_schema() schema, current_version() v").show()


2023-03-16 20:34:34.823 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: Windows-10-10.0.22621-SP0
2023-03-16 20:34:34.823 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-03-16 20:34:35.942 INFO    snowflake.snowpark.session: Snowpark Session information: 
"version" : 1.1.0,
"python.version" : 3.8.16,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 15389347869,
"os.name" : Windows

2023-03-16 20:34:39.617 INFO    snowflake.connector.cursor: query: [select current_warehouse() wh, current_database() db, current_schema() schema, c...]
2023-03-16 20:34:39.771 INFO    snowflake.connector.cursor: query execution done
2023-03-16 20:34:39.771 INFO    snowflake.connector

----------------------------------------------
|"WH"        |"DB"        |"SCHEMA"  |"V"    |
----------------------------------------------
|COMPUTE_WH  |SUMMIT_HOL  |PUBLIC    |7.9.0  |
----------------------------------------------



# Query the data

In [17]:
# SQL query to explore the data
session.sql("SELECT * FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE \"Table Name\" = 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product' AND \"Indicator Name\" = 'Personal consumption expenditures (PCE)' AND \"Frequency\" = 'A' ORDER BY \"Date\"").show()

2023-03-16 20:35:30.521 INFO    snowflake.connector.cursor: query: [SELECT * FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE "Table Name" = 'Price Ind...]
2023-03-16 20:35:30.826 INFO    snowflake.connector.cursor: query execution done
2023-03-16 20:35:30.826 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM (SELECT * FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE "Table N...]
2023-03-16 20:35:33.123 INFO    snowflake.connector.cursor: query execution done


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Table"  |"Table Name"                                        |"Table Description"  |"Table Full Name"                                   |"Table Unit"     |"Indicator"  |"Indicator Name"                         |"Indicator Description"  |"Indicator Full Name"  |"Units"          |"Scale"  |"Frequency"  |"Date"      |"Value"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|T20304   |Pr

In [18]:
# Now use Snowpark dataframe
snow_df_pce = (session.table("ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA") 
                            .filter(col('Table Name') == 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product') 
                            .filter(col('Indicator Name') == 'Personal consumption expenditures (PCE)')
                            .filter(col('"Frequency"') == 'A')
                            .filter(col('"Date"') >= '1972-01-01'))
snow_df_pce.show()

2023-03-16 20:36:19.857 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA]
2023-03-16 20:36:20.170 INFO    snowflake.connector.cursor: query execution done
2023-03-16 20:36:20.170 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE (((("Table Name" = 'Pri...]
2023-03-16 20:36:21.922 INFO    snowflake.connector.cursor: query execution done


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Table"  |"Table Name"                                        |"Table Description"  |"Table Full Name"                                   |"Table Unit"     |"Indicator"  |"Indicator Name"                         |"Indicator Description"  |"Indicator Full Name"  |"Units"          |"Scale"  |"Frequency"  |"Date"      |"Value"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|T20304   |Pr

In [20]:
# Let Snowflake perform filtering using the Snowpark pushdown and display results in a Pandas dataframe
snow_df_pce = (session.table("ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA")
                        .filter(col('"Table Name"') == 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product')
                        .filter(col('"Indicator Name"') == 'Personal consumption expenditures (PCE)')
                        .filter(col('"Frequency"') == 'A')
                        .filter(col('"Date"') >= '1972-01-01'))
pd_df_pce_year = snow_df_pce.select(year(col('"Date"')).alias('"Year"'), col('"Value"').alias('PCE') ).to_pandas()
pd_df_pce_year


2023-03-16 20:36:54.678 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA]
2023-03-16 20:36:54.936 INFO    snowflake.connector.cursor: query execution done
2023-03-16 20:36:54.944 INFO    snowflake.connector.cursor: query: [SELECT year("Date") AS "Year", "Value" AS "PCE" FROM ( SELECT  *  FROM ECONOMY_D...]
2023-03-16 20:36:55.146 INFO    snowflake.connector.cursor: query execution done


SnowparkFetchDataException: (1406): Failed to fetch a Pandas Dataframe. The error is: 255002: 255002: Optional dependency: 'pandas' is not installed, please see the following link for install instructions: https://docs.snowflake.com/en/user-guide/python-connector-pandas.html#installation

# Train the Linear Regression model

In [ ]:
# train model with PCE index

x = pd_df_pce_year["Year"].to_numpy().reshape(-1,1)
y = pd_df_pce_year["PCE"].to_numpy()

model = LinearRegression().fit(x, y)

# test model for 2022
predictYear = 2022
pce_pred = model.predict([[predictYear]])
# print the last 5 years
print (pd_df_pce_year.tail() )
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(pce_pred[0],2)))


### Creating a User Defined Function within Snowflake to do the scoring there

In [ ]:
def predict_pce(predictYear: int) -> float:
    return model.predict([[predictYear]])[0].round(2).astype(float)

_ = session.udf.register(predict_pce,
                        return_type=FloatType(),
                        input_type=IntegerType(),
                        packages= ["pandas","scikit-learn"],
                        is_permanent=True, 
                        name="predict_pce_udf", 
                        replace=True,
                        stage_location="@udf_stage")

# Test the trained model by invoking the UDF via a SQL statement

In [ ]:
session.sql("select predict_pce_udf(2022)").show()
